DenseNet 也是基于resnet跨层连接的思想。 cvpr2017 best paper. 

resnet 是跨层求和， DenseNet 是跨层将特征在通道维度进行拼接。

因为是在通道维度进行特征拼接，多以底层的输出会保留进入所有后边的层， 能够更好的保证梯度的传播，同时能使用低维的特征和高维的特征进行联合训练，能够得到更好的结果。

In [1]:
import tensorflow as tf
import cifar10_input
batch_size = 64
images_trains, labels_trains = cifar10_input.distorted_inputs(data_dir="../cifar_data/cifar-10-batches-bin/",
                                                              batch_size=batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data=True,data_dir="../cifar_data/cifar-10-batches-bin/",
                                               batch_size=batch_size)

import tensorflow.contrib.slim as slim

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Filling queue with 20000 CIF

In [2]:
#基本的卷积单元
def bn_relu_conv(x, out_depth, scope='dense_basic_conv', reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        net = slim.batch_norm(x, activation_fn=None, scope='bn')
        net = slim.nn.relu(net, name='activation')
        net = slim.conv2d(net, out_depth,3, activation_fn=None, normalizer_fn=None,
                          biases_regularizer=None, scope='conv')
        return net

In [3]:
#densenet 基本单元
def dense_block(x, growth_rate, num_layers, scope='dense_block', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope, reuse=reuse):
        net =x
        for i in range(num_layers):
            out = bn_relu_conv(net, growth_rate, scope='block%d' % i)
            net = tf.concat([net, out], axis=-1)
        return net

In [4]:
def transition(x, out_depth, scope='transition', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    with tf.variable_scope(scope, reuse=reuse):
        net = slim.batch_norm(x, activation_fn=None, scope='bn')
        net = tf.nn.relu(net, name='activation')
        net = slim.conv2d(net, out_depth, 1, activation_fn=None, normalizer_fn=None, 
                          biases_regularizer=None, scope='conv')
        net = slim.avg_pool2d(net, 2,2, scope='avg_pool')
        
        return net

In [5]:
def densenet(x, num_classes, growth_rate=32, block_layers=[6, 12, 24, 16], is_training=None, 
             scope='densenet', reuse=None, verbose=False):
    with tf.variable_scope(scope, reuse=reuse):
        with slim.arg_scope([slim.batch_norm], is_training=is_training):
            if verbose:
                print('input: {}'.format(x.shape))
            
            with tf.variable_scope('block0'):
                net = slim.conv2d(x, 64, [7,7], stride=2, normalizer_fn=None, activation_fn=None,
                                  scope='conv_7x7')
                net = tf.nn.relu(net, name='activation')
                net = slim.max_pool2d(net, [3,3], stride=2, scope='max_pool')
                if verbose:
                    print('block0: {}'.format(net.shape))
            #循环构建block transition
            for i, num_layers in enumerate(block_layers):
                with tf.variable_scope('block%d' % (i+1)):
                    net = dense_block(net, growth_rate, num_layers)
                    if i != len(block_layers)-1:
                        current_depth = net.get_shape().as_list()[-1]
                        net = transition(net, current_depth//2)
                if verbose:
                    print('block{}: {}'.format(i+1, net.shape))
                
                with tf.variable_scope('block%d' % (len(block_layers)+1)):
                    net = slim.batch_norm(net, activation_fn=None, scope='bn')
                    net = tf.nn.relu(net,name='activation')
                    net = tf.reduce_mean(net, [1,2], name='global_pool', keep_dims=True)
                    
                    if verbose:
                        print('block{}: {}'.format(len(block_layers)+1, net.shape))  
                with tf.variable_scope('classification'):
                    net = slim.flatten(net, scope='flattern')
                    net = slim.fully_connected(net, num_classes, activation_fn=None, normalizer_fn=None,
                                               scope='logit')
                    if verbose:
                        print('classification: {}'.format(net.shape))
                    return net
                    
                      

In [6]:
with slim.arg_scope([slim.conv2d], activation_fn = tf.nn.relu, normalizer_fn = slim.batch_norm) as sc:
    conv_scope = sc

is_training = tf.placeholder(tf.bool, name='is_training')
image_holder = tf.placeholder(shape=[batch_size,24,24,3], dtype=tf.float32)
label_holder = tf.placeholder(shape=[batch_size], dtype=tf.int32)

with slim.arg_scope(conv_scope):
    train_out = densenet(image_holder,10, is_training=is_training, verbose=True)

with tf.variable_scope('loss'):
    train_loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=train_out,
                                                                       labels=label_holder, scope='train'))

with tf.variable_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1,output_type=tf.int32), 
                                                    label_holder), tf.float32))

lr = 1e-3
opt = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
tf.train.start_queue_runners()

for i in range(1000):
    image_bath, label_batch = sess.run([images_trains, labels_trains])
    _, loss1, acc1 = sess.run([train_op,train_loss,train_acc], feed_dict={image_holder: image_bath, label_holder: label_batch, is_training:True})
    if i % 10 ==0:
        print("Step%d  train loss %.6f train acc: %.6f" % (i+1, loss1, acc1))

input: (64, 24, 24, 3)
block0: (64, 5, 5, 64)
block1: (64, 2, 2, 128)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
block5: (64, 1, 1, 128)
classification: (64, 10)
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Step1  train loss 2.613547 train acc: 0.187500
Step11  train loss 2.326733 train acc: 0.218750
Step21  train loss 2.337852 train acc: 0.140625
Step31  train loss 2.134678 train acc: 0.250000
Step41  train loss 1.977512 train acc: 0.406250
Step51  train loss 2.132381 train acc: 0.265625
Step61  train loss 1.976288 train acc: 0.265625
Step71  train loss 2.107504 train acc: 0.203125
Step81  train loss 1.965948 train acc: 0.343750
Step91  train loss 1.960451 train acc: 0.359375
Step101  train loss 1.824190 train acc: 0.406250
Step111  train loss 1.935184 train acc: 0.296875
Step121  train loss 1.747360 train acc: 0.359375
Step131  train loss 1.815198 train acc: 0.421875
Step141  train loss 2.040463 train acc: 0.343750
S